# 10-4 ImageFolder (2)

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
    torch.cuda.manual_seed_all(777)

In [11]:
# 데이터는 tensor 형태로 변형
trans = transforms.Compose([
    transforms.ToTensor()
])
# 데이터를 가져옴
train_data = torchvision.datasets.ImageFolder(root='./custom_data/train_data', transform=trans)

In [12]:
data_loader = DataLoader(dataset = train_data, batch_size = 8, shuffle = True, num_workers=2)

In [13]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3,6,5),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(6,16,5),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.layer3 = nn.Sequential(
            nn.Linear(16*13*29, 120),
            nn.ReLU(),
            nn.Linear(120,2)
        )
        
    def forward(self, x):
        out = self.layer1(x)
        # print(out.shape)
        out = self.layer2(out)
        # 펼침
        out = out.view(out.shape[0], -1)
        out = self.layer3(out)
        return out
        

In [14]:
#testing 
# print문을 다 걸어놓고 각 layer를 통과할때마다 size가 어떻게 되는지 볼 수 있다
net = CNN().to(device)
test_input = (torch.Tensor(3,3,64,128)).to(device)
test_out = net(test_input)

torch.Size([3, 6, 30, 62])


In [7]:
# Adam optimizer로 학습하고자 함.
optimizer = optim.Adam(net.parameters(), lr=0.00005)
loss_func = nn.CrossEntropyLoss().to(device)

In [8]:
total_batch = len(data_loader)

epochs = 7
for epoch in range(epochs):
    avg_cost = 0.0
    for num, data in enumerate(data_loader):
        imgs, labels = data
        imgs = imgs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        out = net(imgs)
        loss = loss_func(out, labels)
        loss.backward()
        optimizer.step()
        
        avg_cost += loss / total_batch
        
    print('[Epoch:{}] cost = {}'.format(epoch+1, avg_cost))
print('Learning Finished!')   

[Epoch:1] cost = 0.6340207457542419
[Epoch:2] cost = 0.3754148781299591
[Epoch:3] cost = 0.11268715560436249
[Epoch:4] cost = 0.035626836121082306
[Epoch:5] cost = 0.01645025797188282
[Epoch:6] cost = 0.009241115301847458
[Epoch:7] cost = 0.00571438716724515
Learning Finished!


In [16]:
# 그 network의 state를 model 폴더의 model.pth에 저장하도록 한다.
torch.save(net.state_dict(), "./model/model.pth")

In [17]:
# 위에서 net을 선언했던 것처럼 선언해준다.
new_net = CNN().to(device)

In [18]:
# new_net에 가져온 값을 담아라
new_net.load_state_dict(torch.load('./model/model.pth'))

In [19]:
print(net.layer1[0])
print(new_net.layer1[0])

# weight 값의 세번째에 해당하는 것을 뽑아내라
# 기존 모델과 불러온 모델과 값이 똑같은 것을 확인할 수 있다.
print(net.layer1[0].weight[0][0][0])
print(new_net.layer1[0].weight[0][0][0])

net.layer1[0].weight[0] == new_net.layer1[0].weight[0]

Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
tensor([-0.0966, -0.0020, -0.0223, -0.0263,  0.0881], grad_fn=<SelectBackward>)
tensor([-0.0966, -0.0020, -0.0223, -0.0263,  0.0881], grad_fn=<SelectBackward>)


tensor([[[1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1]],

        [[1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1]],

        [[1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1]]], dtype=torch.uint8)

In [20]:
# data가 너무 컸으므로 데이터를 64 x 128로 크기를 바꾸고 tensor로 바꾼다.
trans=torchvision.transforms.Compose([
    transforms.Resize((64,128)),
    transforms.ToTensor()
])
test_data = torchvision.datasets.ImageFolder(root='./custom_data/test_data', transform=trans)

In [21]:
test_set = DataLoader(dataset = test_data, batch_size = len(test_data))

In [22]:
with torch.no_grad():
    for num, data in enumerate(test_set):
        imgs, label = data
        imgs = imgs.to(device)
        label = label.to(device)
        
        prediction = net(imgs)
        
        correct_prediction = torch.argmax(prediction, 1) == label
        
        accuracy = correct_prediction.float().mean()
        print('Accuracy:', accuracy.item())

torch.Size([50, 6, 30, 62])
Accuracy: 0.41999998688697815
